In [8]:
from typing import List
from Dataset.CustomDataset import AgeGroupAndAgeDataset, StandardDataset, AgeGroupAndAgeDatasetKL
from Dataset.CustomDataLoaders import CustomDataLoader
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from Utils import AAR, CSVUtils, AgeConversion
from Utils.Validator import Validator

In [9]:
#Caricamento del dataframe
df = CSVUtils.get_df_from_csv("./training_caip_contest.csv", "./training_caip_contest/")

#Suddivisione del dataframe in 3 age groups
_, label_map = CSVUtils.get_df_with_age_subdivision(df, 3)

In [10]:
df_train, df_val = train_test_split(df, test_size=0.25, random_state=42)
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)

from torchvision import transforms
import torch

transform_func = transforms.Compose([
    transforms.Resize(224),
    transforms.PILToTensor(),
    transforms.ConvertImageDtype(torch.float),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(0.1, 0.1, 0.1, 0.1),
    transforms.RandomGrayscale(),
])

transform_func_val = transforms.Compose([
    transforms.Resize(224),
    transforms.PILToTensor(),
    transforms.ConvertImageDtype(torch.float),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
])

# Implementazione di un Dataset utilizzando "CustomDataset" per l'architettura con Film
cd_train = AgeGroupAndAgeDatasetKL(df_train, path_col="path", label_col="age", label_function="CAE", 
                                   label_map=label_map, label_map_n_classes=3, transform_func=transform_func)

cd_val = StandardDataset(df_val, path_col="path", label_col="age", label_function="CAE", transform_func=transform_func_val)
cd_val.set_n_classes(81)
cd_val.set_starting_class(1)

dm_train = CustomDataLoader(cd_train)
dl_train = dm_train.get_balanced_class_dataloader(class_ranges=[(0, 11), (11, 21), (21, 31), (31, 41), (41, 51), (51, 61), (61, 71), (71, 91)], 
                                                    batch_size=128, num_workers=20, prefetch_factor=4)

In [11]:
from ResNetFilmed.resnet import ResNetFiLMed, BackBone, ResNetNotFiLMed, DoNothingLayer
from torchvision.models import resnet18, ResNet18_Weights
import torch
from torch import optim
import torch.nn.functional as F
from torch import nn

####################################################
EPOCHS = 24
####################################################

backbone = resnet18(ResNet18_Weights.IMAGENET1K_V1)
backbone.fc = DoNothingLayer()
backbone.train()
backbone.requires_grad_(False)
backbone.to("cuda")
model_age = ResNetNotFiLMed(backbone, 81)
opt = optim.SGD(model_age.fc0.parameters(), lr=5e-3, weight_decay=5e-4)
scheduler = optim.lr_scheduler.OneCycleLR(opt, 1e-2, steps_per_epoch=len(dl_train), epochs=EPOCHS)
kl = nn.KLDivLoss(reduction="batchmean")

c:\Users\Gio\.conda\envs\AV_project\lib\site-packages\torchvision\models\_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(


In [12]:
validator = Validator(cd_val, AgeConversion.EVAge, 32, num_workers=8, prefetch_factor=4)

In [13]:
model_age.load_state_dict(torch.load("./model_age_balanced_simple_5__5.87.pt"))

<All keys matched successfully>

In [14]:
best_val_aar = -1

for e in range(EPOCHS):
    with tqdm(dl_train, unit=" batch") as tepoch:
        for batch in tepoch:
            opt.zero_grad()
            x, y = batch
            x = x.to("cuda")
            y_age = y[1].to("cuda")
            y_age_kl: torch.Tensor = y[2].to("cuda")

            out_age = model_age(x)
            loss_age_kl: torch.Tensor = kl(F.log_softmax(out_age, dim=-1), y_age_kl)

            out = F.softmax(out_age, dim=-1)
            out = AgeConversion.EVAge(out)
            y_age = AgeConversion.EVAge(y_age)

            loss = loss_age_kl
            loss.backward()
            opt.step()
            scheduler.step()

            tepoch.set_postfix(loss_age_kl=loss_age_kl.detach().cpu().numpy())

    def forward_function(x):
        out = model_age(x)
        out = F.softmax(out, dim=-1)
        return out

    val_aar, val_aar_old = validator.validate(forward_function)
    print(val_aar, val_aar_old)

    if val_aar > best_val_aar:
        best_val_aar = val_aar
        torch.save(model_age.state_dict(), "./model_age_balanced_simple.pt")
        print("Saved model")

100%|██████████| 4493/4493 [01:45<00:00, 42.48 batch/s]


tensor(4.2791, dtype=torch.float64) tensor(3.8569, dtype=torch.float64)
Saved model


100%|██████████| 4493/4493 [01:47<00:00, 41.92 batch/s]


tensor(4.5046, dtype=torch.float64) tensor(3.9836, dtype=torch.float64)
Saved model


100%|██████████| 4493/4493 [01:46<00:00, 42.09 batch/s]


...

100%|██████████| 4493/4493 [01:49<00:00, 41.12 batch/s]


tensor(4.4113, dtype=torch.float64) tensor(3.8343, dtype=torch.float64)


100%|██████████| 4493/4493 [01:45<00:00, 42.45 batch/s]


tensor(4.2923, dtype=torch.float64) tensor(3.5992, dtype=torch.float64)


 78%|███████▊  | 2624/3369 [11:42<01:56,  6.38 batch/s, loss_age_kl=1.6414823165788877]